In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# VectorMagnitudeComp

`VectorMagnitudeComp` computes the magnitude (L2 norm) of a single input of some given length.
It may be vectorized to provide the result at one or more points simultaneously.

$$
    \lvert a_i \rvert = \sqrt{\bar{a}_i \cdot \bar{a}_i}
$$

## VectorMagnitudeComp Options

The default `vec_size` is 1, providing the magnitude of $a$ at a singlepoint.  The length of $a$ is provided by option `length`.

Other options for VectorMagnitudeComp allow the user to rename the input variable $a$ and the output $a\_mag$, as well as specifying their units.

In [2]:
import openmdao.api as om
om.show_options_table("openmdao.components.vector_magnitude_comp.VectorMagnitudeComp")

Option,Default,Acceptable Values,Acceptable Types,Description
always_opt,False,"[True, False]",['bool'],"If True, force nonlinear operations on this component to be included in the optimization loop even if this component is not relevant to the design variables and responses."
distributed,False,"[True, False]",['bool'],"If True, set all variables in this component as distributed across multiple processes"
in_name,a,N/A,['str'],The variable name for input vector.
length,3,N/A,['int'],The length of the input vector at each point
mag_name,a_mag,N/A,['str'],The variable name for output vector magnitude.
run_root_only,False,"[True, False]",['bool'],"If True, call compute, compute_partials, linearize, apply_linear, apply_nonlinear, and compute_jacvec_product only on rank 0 and broadcast the results to the other ranks."
units,N/A,N/A,['str'],The units of the input vector.
vec_size,1,N/A,['int'],The number of points at which the vector magnitude is computed


## VectorMagnitudeComp Constructor

The call signature for the `VectorMagnitudeComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.vector_magnitude_comp.VectorMagnitudeComp.__init__
        :noindex:
```

## VectorMagnitudeComp Usage

There are often situations when numerous magnitudes need to be computed, essentially in parallel.
You can reduce the number of components required by having one `VectorMagnitudeComp` perform multiple operations.
This is also convenient when the different operations have common inputs.

The `add_magnitude` method is used to create additional magnitude calculations after instantiation.

```{eval-rst}
    .. automethod:: openmdao.components.vector_magnitude_comp.VectorMagnitudeComp.add_magnitude
       :noindex:
```

## VectorMagnitudeComp Example

In the following example VectorMagnitudeComp is used to compute the radius vector magnitude
given a radius 3-vector at 100 points simultaneously. Note the use of `in_name` and `mag_name` to assign names to the inputs and outputs. Units are assigned using `units`.  The units of the output magnitude are the same as those for the input.

In [3]:
import numpy as np
import openmdao.api as om

n = 100

p = om.Problem()

comp = om.VectorMagnitudeComp(vec_size=n, length=3,
                              in_name='r', mag_name='r_mag', units='km')

p.model.add_subsystem(name='vec_mag_comp', subsys=comp,
                      promotes_inputs=[('r', 'pos')])

p.setup()

p.set_val('pos', 1.0 + np.random.rand(n, 3))

p.run_model()

# Verify the results against numpy.dot in a for loop.
expected = []
for i in range(n):
    a_i = p.get_val('pos')[i, :]
    expected.append(np.sqrt(np.dot(a_i, a_i)))

    actual_i = p.get_val('vec_mag_comp.r_mag')[i]
    rel_error = np.abs(expected[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

print(p.get_val('vec_mag_comp.r_mag'))

[2.47720613 3.22427911 2.79689459 2.76126771 2.66406024 2.93932592
 2.31476852 2.89726892 2.7932659  2.54740269 2.66226257 2.46339129
 2.45432278 2.91100245 2.72329421 2.51829663 2.86398109 2.95142393
 2.57617835 2.2135368  2.88943274 2.98101329 3.2244157  2.09997384
 2.81862124 2.30816528 2.65774379 2.72487515 2.46078437 2.45414194
 2.5180635  3.15831824 2.5536861  2.65733062 2.39147064 2.49045354
 3.04687889 2.75837572 2.10911018 2.32916248 2.49915555 2.70020097
 2.37949724 3.01096739 2.84609002 2.69292036 3.19798898 1.95889032
 3.17428981 2.94502472 2.63789937 2.9944546  2.09142853 2.09498564
 2.72746127 2.46575456 2.74604231 2.37930903 2.76795827 2.47367081
 2.75793952 2.90696274 2.48960749 2.90293261 2.74475228 2.425904
 2.65404569 2.17779336 2.43371864 2.80590763 2.36198111 2.19294104
 2.63185844 2.75045122 2.964557   3.1236848  2.82982798 2.60565878
 2.98022372 2.67138663 2.70882965 2.78307888 2.68482528 2.81350395
 2.59108427 2.09473765 2.17655328 3.23326821 2.8207744  2.860555

In [4]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('vec_mag_comp.r_mag'), np.array(expected))

6.206938993434361e-17

## VectorMagnitudeComp Example with Multiple Magnitudes

Note that, when defining multiple magnitudes, an input name in one call to `add_magnitude` may not be an output name in another call, and vice-versa.

In [5]:
import numpy as np

n = 100

p = om.Problem()

comp = om.VectorMagnitudeComp(vec_size=n, length=3,
                              in_name='r', mag_name='r_mag', units='km')

comp.add_magnitude(vec_size=n, length=3,
                   in_name='b', mag_name='b_mag', units='ft')

p.model.add_subsystem(name='vec_mag_comp', subsys=comp,
                      promotes_inputs=['r', 'b'])

p.setup()

p.set_val('r', 1.0 + np.random.rand(n, 3))
p.set_val('b', 1.0 + np.random.rand(n, 3))

p.run_model()

# Verify the results against numpy.dot in a for loop.
expected_r = []
expected_b = []
for i in range(n):
    a_i = p.get_val('r')[i, :]
    expected_r.append(np.sqrt(np.dot(a_i, a_i)))

    actual_i = p.get_val('vec_mag_comp.r_mag')[i]
    rel_error = np.abs(expected_r[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

    b_i = p.get_val('b')[i, :]
    expected_b.append(np.sqrt(np.dot(b_i, b_i)))

    actual_i = p.get_val('vec_mag_comp.b_mag')[i]
    rel_error = np.abs(expected_b[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

print(p.get_val('vec_mag_comp.r_mag'))

[2.16909513 2.31484139 2.1553754  2.26823389 2.62191047 2.69089259
 2.92511147 2.81172221 2.88230408 2.99339444 2.80249834 3.17668009
 3.30847843 2.94933918 2.14980714 2.68058423 2.99421523 2.46476956
 2.99592247 2.39167247 2.9381786  2.45940609 2.00891224 3.24347441
 2.11641843 2.66017763 2.43954382 2.59119938 2.56812034 2.00580092
 2.25745868 2.32984385 2.27979531 2.65531736 2.98510633 2.71093903
 2.55121194 2.89978764 2.65676425 2.75940445 3.20361578 2.61292702
 2.69709788 2.55761854 2.4688588  2.25872174 2.4873349  2.74086404
 2.27145181 2.42832141 2.82401819 2.25611018 3.11158394 2.5596496
 2.65985986 2.40461125 2.04029604 2.45138562 2.33155441 2.82890186
 2.90370387 2.69566135 3.09228606 3.26733392 2.65650134 2.63552336
 2.94427179 2.910518   2.59693374 2.59146563 2.42203945 2.61999688
 2.63347728 2.57154506 2.53671846 2.59376095 2.59784258 2.99502527
 2.8462322  2.20084296 2.49992886 2.54874362 2.65832316 2.87099382
 2.38961611 2.5853811  2.40343661 2.45445522 2.0560535  2.30189

In [6]:
print(p.get_val('vec_mag_comp.b_mag'))

[2.50569687 2.69284146 2.52691302 3.2708491  2.83961974 2.53759691
 2.76146508 2.76474766 2.41099842 2.1752986  2.39602338 3.15424525
 1.93374109 1.94382857 2.64499855 2.15168531 2.63849411 2.27947939
 3.08403152 2.82203469 3.22401139 2.47380494 2.26321186 2.8875496
 2.56771933 2.60151038 2.3058943  2.77524314 2.63831428 2.34649444
 2.56401134 2.70933573 2.62342509 2.7813774  2.67102014 2.46713211
 2.57825045 2.36867586 3.15058892 3.02129469 2.40320863 1.89942062
 2.48782702 2.45208937 2.29927693 2.83513115 2.7742489  3.0032569
 2.93768852 3.09179415 2.44028556 2.92920553 2.68855166 2.02035801
 2.72780915 2.76203803 2.95242611 2.31933228 2.22646588 2.67490904
 2.72177567 2.24960096 2.94634973 2.82528482 3.11303558 2.66744558
 3.30195065 2.77466685 2.33276246 3.03439913 2.46957745 2.32666467
 2.37320572 2.64207015 2.75884949 2.30902782 2.20478052 2.57885685
 2.75971356 2.710412   2.36083182 2.6957311  2.7071677  2.36035488
 3.05998955 2.98277305 2.87554915 2.70009616 2.72004103 2.319833

In [7]:
assert_near_equal(p.get_val('vec_mag_comp.r_mag'), np.array(expected_r))
assert_near_equal(p.get_val('vec_mag_comp.b_mag'), np.array(expected_b))

5.325068629033931e-17